# LAVA between ability to smell and PD

by Jeff Kim

----

**Project:** Bidirectional relationship between anosmia and Parkinson's disease

**Version:** R/4.1

**Status:** COMPLETE

**Last Updated:** JUNE-2024

## Notebook Overview

This notebook will show the code used to run global genetic correlation between PD and ability to smell.

---

# 1. Set up Workspace

## 1a. Load packages and set working directory

In [1]:
BiocManager::install("snpStats")
devtools::install_github("https://github.com/josefin-werme/LAVA.git", build_vignettes=TRUE)

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.17 (BiocManager 1.30.22), R 4.3.0 (2023-04-21)

Warning message:
“package(s) not installed when version(s) same as or greater than current; use
  `force = TRUE` to re-install: 'snpStats'”
Installation paths not writeable, unable to update packages
  path: /usr/local/apps/R/4.3/4.3.0/lib64/R/library
  packages:
    KernSmooth, foreign, mgcv, nlme, spatial, survival
  path: /usr/local/apps/R/4.3/site-library_4.3.0
  packages:
    CCA, ChIPQC, DT, DelayedMatrixStats, DescTools, DiffBind, EMCluster, FSA,
    GenomeInfoDb, GenomicDataCommons, GenomicFeatures, Gmisc, GreyListChIP,
    HSAUR2, Hmisc, ICS, Matrix, RNOmni, ROpenCVLite, Rcmdr, RcmdrMisc,
    RcppArmadillo, RcppNumerical, Rfit, Rhdf5lib, S4Arrays, Seurat,
    StanHeaders, Surrogate, arrow, askpass, biocViews, blockm

keep         (NA -> 1.0  ) [CRAN]
matrixsam... (NA -> 2.0.0) [CRAN]


Skipping 1 packages not available: snpStats

Installing 2 packages: keep, matrixsampling

Installing packages into ‘/gpfs/gsfs9/users/kimjoj/R/rhel8/4.3’
(as ‘lib’ is unspecified)



── R CMD build ───────────────────────────────────────────────────────────────────
✔  checking for file ‘/lscratch/7954970/Rtmp947kbo/remotesf3744bbf1a1b/josefin-werme-LAVA-5b2adc7/DESCRIPTION’
─  preparing ‘LAVA’:
✔  checking DESCRIPTION meta-information
─  installing the package to build vignettes
✔  creating vignettes (5s)
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
   Omitted ‘LazyData’ from DESCRIPTION
─  building ‘LAVA_0.1.0.tar.gz’
   


Installing package into ‘/gpfs/gsfs9/users/kimjoj/R/rhel8/4.3’
(as ‘lib’ is unspecified)



In [ ]:
libraries <- c("data.table", "tidyverse", "GenomicRanges", "qdapTools", "stringr", "rtracklayer", "LAVA")
lapply(libraries, require, character.only = TRUE)

In [ ]:
setDTthreads(7)
"%+%" = function(...) paste0(...,sep="")

# 2. Prepare prerequisite files

The required files:

1. PLINK reference file - provided by LAVA
2. Loci blocks - provided by LAVA
3. Table of traits analyzed
4. Sample overlap (optional)
5. Summary statistics

## 2a. Table of traits analyzed

-   **Input info file**, used for convenient processing of multiple
    phenotypes. Requires the columns:

    -   ***phenotype***: phenotype IDs

    -   ***cases***: number of cases (set to NA for continuous
        phenotypes)

    -   ***controls:*** number of controls (set to NA for continuous
        phenotypes)

    -   ***prevalence*** (optional): the population prevalence of binary
        phenotypes

        -   this is only relevant if you want an estimate of the local
            population *h*<sup>2</sup> for binary phenotypes. Estimates
            of the observed local sample *h*<sup>2</sup> are still
            provided

    -   ***filename***: paths and file names to the relevant summary
        statistics

In [ ]:
studies_PD_smell <- data.table(
    phenotype = c('ability_to_smell', 'PD'),
    cases = c(NA, 15056),
    controls = c(NA, 12637),
    pop_prevalence = c(NA, 0.005),
    path = c('data/ldsc_munged_sumstats/PD_clinical.sumstats.gz', "data/ldsc_munged_sumstats/ability_to_smell_ldsc.sumstats.gz")
)

In [ ]:
fwrite(
    studies_PD_smell_final,
    "phenotype_list/phenotypes_prevalence_PDsmell.LAVA.txt",
    sep = '\t',
    na = "NA",
    quote = F
)

## 2b. Loci blocks

Required columns are LOC, CHR, START, STOP

Shamelessly borrowed from Regina Reynolds: https://github.com/RHReynolds/neurodegen-psych-local-corr

Two types will be generated: all LD blocks and blocks with significant variants in any of the GWAS.

### All LD blocks

In [ ]:
ld_blocks <- fread("refdata/LAVA/LAVA_s2500_m25_f1_w200.blocks")

In [ ]:
ld_blocks_all <- ld_blocks
ld_blocks_all <- ld_blocks_all %>% rename(
    "chr"="CHR",
    "start"="START",
    "stop"="STOP"
)
ld_blocks_all <- ld_blocks_all[, LOC := 1:nrow(ld_blocks_all)]
head(ld_blocks_all)

In [ ]:
ld_blocks_all <- ld_blocks_all[, c("LOC","CHR","START","STOP")]
fwrite(
    ld_blocks_all,
    sep = "\t",
    file = paste0("refdata/LAVA/loci_blocks/all/EUR.all_blocks.loci")
)
for (i in 1:22) {
    fwrite(
        ld_blocks_all[CHR==i],
        sep = "\t",
        file = paste0("refdata/LAVA/loci_blocks/all/EUR.all_blocks.CHR", i, ".loci")
    )
}

### GWAS significant blocks

In [ ]:
studies_PD_smell_final <- fread("phenotype_list/phenotypes_prevalence_PDsmell.LAVA.txt")

In [ ]:
gwas_list <-
  setNames(
    object = list(
      fread("data/formatted_sumstats/ability_to_smell_formatted.tsv.gz"),
      fread("data/formatted_sumstats/PD_formatted.tsv.gz")
    ),
    nm = c("ability_to_smell", "PD")
  )

In [ ]:
ref <- import("refdata/Homo_sapiens.GRCh37.87.gtf")
ref <- ref %>% keepSeqlevels(c(1:22), pruning.mode = "coarse")
ref <- ref[ref$type == "gene"]

In [ ]:
# Main --------------------------------------------------------------------

# Filter for only genome-wide significant loci and convert to granges
gr_list <-
  gwas_list %>%
  lapply(., function(gwas){
    gwas %>%
      dplyr::filter(P < 5e-8) %>%
      GenomicRanges::makeGRangesFromDataFrame(
        .,
        keep.extra.columns = TRUE,
        ignore.strand = TRUE,
        seqinfo = NULL,
        seqnames.field = "CHR",
        start.field = "BP",
        end.field = "BP"
      )
  })

# Add locus id and rename remaining columns to fit LAVA requirements
ld_blocks <-
  ld_blocks %>%
  dplyr::rename_with(
    .fn = stringr::str_to_upper,
    .cols = everything()
  ) %>%
  dplyr::mutate(
    LOC = dplyr::row_number()
  ) %>%
  dplyr::select(LOC, everything())

# Convert to granges
ld_blocks_gr <-
  ld_blocks %>%
  GenomicRanges::makeGRangesFromDataFrame(
    .,
    keep.extra.columns = TRUE,
    ignore.strand = TRUE,
    seqinfo = NULL,
    seqnames.field = "chr",
    start.field = "start",
    end.field = "stop"
  )

# Overlap granges objects
overlap_list <-
  gr_list %>%
  lapply(., function(gr){
    GenomicRanges::findOverlaps(gr, ld_blocks_gr, type = "within") %>%
      as_tibble()
  })


# Extract relevant rows from ld_blocks using overlap indices
# Rename remaining columns to fit with LAVA requirements
loci <-
  ld_blocks %>%
  dplyr::slice(
    overlap_list %>%
      qdapTools::list_df2df(col1 = "gwas") %>%
      .[["subjectHits"]] %>%
      unique()
    ) %>%
  dplyr::arrange(LOC)

In [ ]:
out_dir <- "refdata/LAVA"
fwrite(
  loci,
  sep = "\t",
  file = file.path(out_dir, "gwas_filtered.loci")
  )

In [ ]:
# Generate df of GWAS loci that overlap which LD blocks
overlap_df_list <- vector(mode = "list", length = length(gr_list))

for(i in 1:length(gr_list)){

  gr <- gr_list[[i]]

  names(overlap_df_list)[i] <- names(gr_list)[i]

    overlap_df_list[[i]] <-
      gr %>%
      as_tibble() %>%
      dplyr::mutate(
        BETA = NA
      ) %>%
      dplyr::select(seqnames, start, end, SNP, A1, A2, MAF, BETA, SE, P, N)

  overlap_df_list[[i]] <-
    overlap_df_list[[i]] %>%
    dplyr::rename_with(
      ~ stringr::str_c("GWAS", .x, sep = "_")
    ) %>%
    dplyr::slice(overlap_list[[i]]$queryHits) %>%
    dplyr::bind_cols(
      ld_blocks_gr %>%
        as_tibble() %>%
        dplyr::rename_with(
          ~ stringr::str_c("LD", .x, sep = "_")
        ) %>%
        dplyr::slice(overlap_list[[i]]$subjectHits)
    ) %>%
    dplyr::select(-contains("strand")) %>%
    dplyr::rename_with(
      ~ stringr::str_replace(.x,
                             pattern = "seqnames",
                             replacement = "CHR"),
      .col = dplyr::ends_with("seqnames"))

}

overlap_df <-
  overlap_df_list %>%
  qdapTools::list_df2df(col1= "GWAS")

# Generate df of associated GWAS and genes that overlap investigated LD blocks
loci_gr <-
  loci %>%
  GenomicRanges::makeGRangesFromDataFrame(
    .,
    keep.extra.columns = TRUE,
    ignore.strand = TRUE,
    seqinfo = NULL,
    seqnames.field = "CHR",
    start.field = "START",
    end.field = "STOP"
  )

overlap <-
  GenomicRanges::findOverlaps(loci_gr, ref) %>%
  tibble::as_tibble()

loci_genes <-
  tibble::tibble(
    locus = loci_gr[overlap$queryHits]$LOC,
    chr = loci_gr[overlap$queryHits] %>% GenomeInfoDb::seqnames() %>% as.character(),
    locus_start = loci_gr[overlap$queryHits] %>% BiocGenerics::start(),
    locus_end = loci_gr[overlap$queryHits] %>% BiocGenerics::end(),
    gene_id = ref[overlap$subjectHits]$gene_id,
    gene_name =
      ref[overlap$subjectHits]$gene_name %>%
      stringr::str_replace_all("-", "_") %>%
      stringr::str_replace_all("\\.", "_")
  ) %>%
  dplyr::group_by(locus, chr, locus_start, locus_end) %>%
  dplyr::summarise(
    n_overlapping_genes = n(),
    overlapping_gene_id = list(gene_id),
    overlapping_gene_name = list(gene_name)
      )

loci_genes_gwas_df <-
  loci_genes %>%
  dplyr::inner_join(
    overlap_df %>%
      dplyr::distinct(GWAS, LD_LOC) %>%
      dplyr::group_by(LD_LOC) %>%
      dplyr::summarise(
        n_assoc_gwas = n(),
        assoc_gwas = list(str_to_upper(GWAS))
        ),
    by = c("locus" = "LD_LOC")
  )

# Save data ---------------------------------------------------------------

out_dir <- "refdata/LAVA"
fwrite(
  loci,
  sep = "\t",
  file = file.path(out_dir, "gwas_filtered.loci")
  )
saveRDS(
  overlap_df,
  file = file.path(out_dir, "gwas_filtered_loci.rds")
  )
saveRDS(
  loci_genes_gwas_df,
  file = file.path(out_dir, "gwas_filtered_loci_w_genes.rds")
)

In [ ]:
for (i in unique(loci$CHR)) {
    fwrite(
        loci[CHR==i],
        sep = "\t",
        file = paste0(out_dir, "/gwas_filtered.CHR", i, ".loci")
    )
}

# 2. Run LAVA

To run in parallel per chromosome, an R script and bash scripts were made. Each instance were assigned used ~5Gb of ram and it took less than 15 minutes to run with 1 core.

For chromosome 1, the bash script:

R script (`LAVA_script.R`)

# 3. After the run

In [ ]:
temp <- paste0("results/LAVA/sigLoci/PDsmell.CHR", 1, ".bivar.lava.RDS") %>% readRDS()

In [ ]:
for (i in 1:22) {
    temp1 <- paste0("results/LAVA/sigLoci/PDsmell.CHR", i, ".bivar.lava.RDS") %>% readRDS()
    tempDT1 <- rbindlist(temp1)
    temp2 <- paste0("results/LAVA/sigLoci/PDsmell.CHR", i, ".univ.lava.RDS") %>% readRDS()
    tempDT2 <- rbindlist(temp2, fill = T)
    if (i == 1) {
        LavaBivResDT <- tempDT1
        LavaUniResDT <- tempDT2
    } else {
        LavaBivResDT <- rbind(LavaBivResDT, tempDT1, fill = T)
        LavaUniResDT <- rbind(LavaUniResDT, tempDT2, fill = T)
    }
}

In [ ]:
fwrite(
    LavaBivResDT,
    "results/LAVA/sigLoci/aggregate.PDsmell.bivar.lava.txt",
    sep = '\t'
)
fwrite(
    LavaUniResDT,
    "results/LAVA/sigLoci/aggregate.PDsmell.univ.lava.txt",
    sep = '\t'
)

## 3a. Check valid loci using univariate results

In [ ]:
LavaUniResDT <- fread("results/LAVA/sigLoci/aggregate.PDsmell.univ.lava.txt")

Let's remove MHC region

MHC region: 6:28477797-33448354


In [ ]:
LavaUniResDT <- LavaUniResDT[!(chr == 6 & start >= 28477797 & stop <= 33448354)]

In [ ]:
LavaUniResDT <- LavaUniResDT[phen == "PD", P_BONF := p*409]
LavaUniResDT <- LavaUniResDT[phen == "ability_to_smell", P_BONF := p*409]
PD_heritable_loci <- LavaUniResDT[phen == "PD" & P_BONF < 0.05]$locus
smell_heritable_loci <- LavaUniResDT[phen == "ability_to_smell" & P_BONF < 0.05]$locus

In [ ]:
smell_PD_loci <- smell_heritable_loci[smell_heritable_loci %in% PD_heritable_loci]

## 3b Search valid loci in 

In [ ]:
LavaBivarResDT <- fread("results/LAVA/sigLoci/aggregate.PDsmell.bivar.lava.txt")

In [ ]:
LavaBivarResDT_smell_PD <- LavaBivarResDT[phen1 == "ability_to_smell" & phen2 == "PD" & locus %in% smell_PD_loci]
LavaBivarResDT_smell_PD <- LavaBivarResDT_smell_PD[, P_BONF := p.adjust(p, method = "bonferroni")]

In [ ]:
LavaBivarResDT_smell_PD[P_BONF < 0.05]

In [ ]:
fwrite(
    LavaBivarResDT_smell_PD,
    "results/LAVA/sigLoci/aggregate.PDsmell.bivar.validLoci.BONF.lava.txt",
    sep = '\t',
    quote = F,
    na = 'NA'
)

In [ ]:
loci_genes_gwas_df <- readRDS("refdata/LAVA/gwas_filtered_loci_w_genes.rds") %>% as.data.table()

In [ ]:
loci_genes_gwas_df <- loci_genes_gwas_df[,c("locus", "n_overlapping_genes", "overlapping_gene_id", "overlapping_gene_name", "n_assoc_gwas", "assoc_gwas")]

In [ ]:
LavaBivarResDT_annotated <- merge(LavaBivarResDT_wBonf, loci_genes_gwas_df)

In [ ]:
fwrite(
    LavaBivarResDT_annotated,
    "results/LAVA/sigLoci/PDSmell.bivar.annotated.txt",
    sep = '\t',
    na = 'NA',
    quote = F
)